# FLARE Logging

This tutorial covers how to configure logging in FLARE for different use cases and modes.

To learn more, see our [Logging Configuration Documentation](https://nvflare.readthedocs.io/en/main/user_guide/configurations/logging_configuration.html) for a more comprehensive description of the various features.

## Setup

The NVFlare [Getting Started Guide](https://nvflare.readthedocs.io/en/main/getting_started.html) provides instructions for setting up FLARE on a local system or in a Docker image.  We've also cloned the NVFlare GitHub in our top-level working directory.

## Simulator Logging

To get started, let's run the hello-numpy-sag job in the simulator and take a look at the default logging output:

In [ ]:
!mkdir -p hello-numpy-sag-workspace
!nvflare simulator -w hello-numpy-sag-workspace -n 2 -t 2 ../hello-world/hello-numpy-sag/jobs/hello-numpy-sag

Notice how the output contains lots of logs from both the FLARE system, as well as the training workflow.
Additionally, the different level of logs (eg. INFO, WARNING, ERROR) have different console colors.

We can view the default configuration used in this run and the generated log files in the workspace:

In [ ]:
!tree hello-numpy-sag-workspace

### Default Log Config

The log_config.json is the default logging configuration used. 
This configuration comes with pre-configured handlers for console level colors, logs, error logs, structured json logs, and fl training logs.

- The three main sections (formatters, filters, handlers), take care of formatting the structure of the logs, filtering the logs, and sending logs to a destination respectively.
- Under loggers, we configure the root logger and add the handlers

Let's take a look at the configuration under the server (will be the same for all sites by default):

In [ ]:
!cat hello-numpy-sag-workspace/server/local/log_config.json

### Default Log Files

Next we can look at the various log files that are created by the FileHandlers:

- logFileHandler with baseFormatter to write all logs to log.txt
- jsonFileHandler with jsonFormatter to write json formatted logs to log.json
- errorFileHandler with baseFormatter and level “ERROR” to write error level logs to log_error.txt
- FLFileHandler with baseFormatter and FLFilter to write fl training and custom logs to log_fl.txt

Feel free to also check out the site logs as well.

In [ ]:
!cat hello-numpy-sag-workspace/server/log.txt

In [ ]:
!cat hello-numpy-sag-workspace/server/log.json

In [ ]:
!cat hello-numpy-sag-workspace/server/log_error.txt

In [ ]:
!cat hello-numpy-sag-workspace/server/log_fl.txt

### Customization

The log config file can easily be customized for different use cases.

For this example, let's assume we are a federated learning researcher mainly interested in the algorithm parts of the log.
By default, we provide the FLFileHandler which uses the FLFilter to generate the log_fl.txt However we would also like to have a cleaner output in the console, as well as remove/shorten some unnecessary log fields.

### custom_log_config.json

Let's change the consoleHandler accordingly to achieve our goal of a cleaner console output.

Remove the fl_ctx field, and add the identity field for a cleaner log structure:

`"fmt": "%(asctime)s - %(identity)s - %(name)s - %(levelname)s - %(message)s",`

Configure the datefmt to only use seconds rather than milliseconds:

``"datefmt": "%Y-%m-%d %H:%M:%S"``

Add the FLFilter, which only allows logs related to FL training to pass through:

``"filters": ["FLFilter"],``

Say we want to highlight the NPModelPersistor, we can color it blue to make it stand out in the console:
```
"logger_colors": {
    "NPModelPersistor": "blue"
}
```



In [ ]:
!cat custom_log_config.json

In [ ]:
!diff custom_log_config.json hello-numpy-sag-workspace/server/local/log_config.json

Now let's run the simulator with the custom_log_config.json using the ``-l`` option:

In [ ]:
!nvflare simulator -w hello-numpy-sag-workspace -n 2 -t 2 -l custom_log_config.json ../hello-world/hello-numpy-sag/jobs/hello-numpy-sag

Compare this to the original output from the first command, and note the differences in the log output.

In addition to the consoleHandler, the various formatters, filters, other handlers, and loggers can all also be customized.

See the [Logging Configuration Documentation](https://nvflare.readthedocs.io/en/main/user_guide/configurations/logging_configuration.html) for more information on how to customize the different sections.

## Provisioned System and Logging Configuration Commands

For this part of the example, we will showcase how to use the [Dynamic Logging Configuration Commands](https://nvflare.readthedocs.io/en/main/user_guide/configurations/logging_configuration.html#dynamic-logging-configuration-commands) with a running FLARE system.

To provision and start an FL system, you can use [POC mode](setup_poc.ipynb) to quickly get started. Feel free to use an existing **provisioned** FLARE project if you have that available. Remember that we recommend starting the system in a separate terminal. 

Once the system is running and you have logged into the admin console you are ready to try out the commands. We provide two admin commands to enable users to dynamically configure the site or job level logging:

### configure_site_log

Configures the site level logs, but does not affect the job level logs.

``configure_site_log target config``

- **target**: server, client <clients>..., or all
- **config**: log configuration
    - path to a json log configuration file (/path/to/my_log_config.json)
    - log level name/number (debug, INFO, 30)
    - read the current log configuration file (reload)

Try and experiment with the following commands in the admin console:

``configure_site_log server debug``

``configure_site_log client site-1 debug``

``configure_site_log all info``

### configure_job_log

Configures the job logs, does not affect site logs.

``configure_job_log job_id target config``

- **job_id**: id of a running job
- **target**: server, client <clients>..., or all
- **config**: log configuration (see above)

Submit a job with ``submit_job <path>/<to>/<job_folder>``,
then try and experiment with the following commands in the admin console:


``configure_job_log <job_id> server debug``

``configure_job_log <job_id> client site-1 debug``

``configure_job_log <job_id> all info``

``configure_job_log <job_id> all <path>/<to>/custom_log_config.json``

Lastly, take a look at the generated log files in the workspace, noting the difference between the site and job logs.
For example if using POC mode:

In [ ]:
!tree hello-numpy-sag-workspace